# **FOREWORD**

Hello everyone, <br>
We are almost into the half way point in the competition as on date! I hope this kernel finds you learning well and augmenting your knowlwdge hereby! <br>

We have witnessed a lot of good public work in the competition that provides a lot of insights into feature engineering and a lot of useful information too. Thanks to the contributors across the forums! <br> 

I notice that all of these kernels use lambda functions in the count vectorizer and TFIDF vectorizer, thus not being able to save them in the standard pickle format. As a result, these kernels resort to building the entire preprocessing pipeline in the train data while inferring and predicting the test set. This is superfluous and in my opinion, an opportunity cost. <br> 

We do have a solution for this and is presented below.. <br> 

# **IMPORT**

In [ ]:
%%time 

import os

try:
    os.mkdir("/kaggle/working/Cloudpickle");
except:
    pass;

!pip -q download cloudpickle -d "/kaggle/working/Cloudpickle";

print();

# **IMPLEMENTATION**

In [ ]:
%%time 

!pip install -q "/kaggle/working/Cloudpickle/cloudpickle-3.0.0-py3-none-any.whl"
import cloudpickle

import pandas as pd, numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from IPython.display import clear_output
from gc import collect

from tqdm.notebook import tqdm

We shall illustrate an example for this package using the vectorizers defined in the public datasets and kernels. This is an illustration notebook. Feel free to use this as input for your kernels, including the cloudpickle installation folder! <br>
Best wishes!


In [ ]:
%%time 

train = pd.read_csv(f"/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv",)
test  = pd.read_csv(f"/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv",)

In [ ]:
%%time 

myvec = \
[Pipeline(steps = [("Vec", 
                    TfidfVectorizer(tokenizer=lambda x: x,
                                    preprocessor=lambda x: x,
                                    token_pattern=None,
                                    strip_accents='unicode',
                                    analyzer = 'word',
                                    ngram_range=(3,6),
                                    min_df=0.05,
                                    max_df=0.95,
                                    sublinear_tf=True,
                                   )
                   )
                  ]
         ),
 
 Pipeline(steps = [("CVec", 
                    CountVectorizer(
                        tokenizer=lambda x: x,
                        preprocessor=lambda x: x,
                        token_pattern=None,
                        strip_accents='unicode',
                        analyzer = 'word',
                        ngram_range=(2,3),
                        min_df=0.10,
                        max_df=0.85,
                    )
                   )
                  ]
         ), 
];



In [ ]:
%%time 

mysavedvec = [];

for i, xform in tqdm(enumerate(myvec)):
    df = \
    pd.DataFrame(xform.fit_transform([i for i in train['full_text']]).\
                 toarray()
                ).\
    add_prefix(f"vec{i}_")
    
    # Saving the vectorizer   
    mysavedvec.append(xform)
    print(f"Vectorizer {i} sample output - {df.shape}")


Let us save the vectorizer now. We use the cloudpickle library for the same. <br>
We create a folder named **SavedVec** in the working folder and dump the fitted vectorizers there <br>

In [ ]:
%%time 

try:
    os.mkdir(f"/kaggle/working/SavedVec")
except: 
    pass
with open("/kaggle/working/SavedVec/mysavedvec", "wb") as f:
    cloudpickle.dump(mysavedvec, f)
    
print(f"---> Saved the vectorizers")

Now, we can load these vectorizers from our saved path and use them in the test set <br>

# **INFERENCE**

In [ ]:
%%time 

with open("/kaggle/working/SavedVec/mysavedvec", "rb") as f:
    myloadedvec = cloudpickle.load(f)
    
display(myloadedvec)

In [ ]:
%%time 

for i, xform in tqdm(enumerate(myloadedvec)):
    df = \
    pd.DataFrame(xform.transform([i for i in test['full_text']]).\
                 toarray()
                ).\
    add_prefix(f"vec{i}_")
    print(f"Vectorizer {i} sample output = {df.shape}")
    
print();

# **OUTRO**

We are done here! Hope you liked the idea! <br>
All the best for the challenge! <br>

Please feel free to use this and save your inference time for better work! <br>